<a href="https://colab.research.google.com/github/Lexie-v2/Goggle-CO/blob/main/Redes_Neuronales_Artificiales_Vera_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introducción a Keras: Implementando una RNA para clasificar imagenes

IMPLEMENTACIÒN DE UNA RED NEURONAL ARTIFICIAL

##0. Importaciòn de librerias

Como primer paso, vamos a importar las librerias especificas necesarias pra la implementaciòn de la Red Neuronal Artificial, en este caso usaremos el framework **Keras**

Procedemos con la instalaciòn de Keras mediante el paquete de **keras-core**

In [ ]:
#!pip install keras-core

Con la nueva version de Keras, podemos escoger que backend vamos a utilizar, debemos tener en cuenta que por defecto se selecciona **tensorflow**, sin embargo, tenemos disponibles otros backends como **pytorch** o **jax**.
No olvidar que el backend debe configurarse antes de importar Keras y no puede cambiarse despues de la importaciòn.

In [ ]:
#import os
#os.environ["KERAS_BACKEND"] = "torch"

Importamos Keras con el backend seleccionado

In [ ]:
# import keras_core

Otra opcion es utilizar Keras desde el framework Tensorflow. Si el backend que vamos a utilizar es Tensorflow, esta opcion està màs optimizada y suele proporcionar mejores resultados.

Tanto Keras-core como la versión de Keras que se encuentra en framework Tensorflow son totalmente compatibles y no requieren modificaciones en la implementación del programa..

In [1]:
from tensorflow import keras
#Verificamos si el backend (Tensorflow) està utilizando la GPU
import tensorflow as tf
tf.test.gpu_device_name()

''

##1. Preparacion del conjunto de datos de entrenamiento

Descargamos el conjunto de datos de imagenes. En este caso practico vamos a ahacer uso de conjunto de datos "Dogs vs Cats" de Kaggle.

El conjunto de datos contiene 25000 imagenes de perros y gatos. El objetivo es entrenar una RNA profunda utilizando estos rchivos para que sea capaz de clasificar nuevas imagenes en funciòn de si aparece un perro y un gato.

In [2]:
# Descargamos el conjunto de datos en el almacenamiento temporal de Google Colab
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
# Descomprimimos el conjunto de datos
!unzip -q kagglecatsanddogs_5340.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  82.4M      0  0:00:09  0:00:09 --:--:--  102M
